{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# YOLOv8 Training - Banking Detection Model\n",
    "\n",
    "Train a custom object detection model on your exported dataset.\n",
    "\n",
    "**Instructions:**\n",
    "1. Click Runtime > Change runtime type > Select GPU (T4 is free!)\n",
    "2. Upload your banking_dataset zip file when prompted\n",
    "3. Run all cells in order\n",
    "4. Download trained model when complete"
   ]
  },

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install ultralytics -q

# Verify installation
import torch
from ultralytics import YOLO
import os

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ CUDA version: {torch.version.cuda}")
print(f"✅ Ultralytics installed successfully!")

## 📤 Step 2: Upload Your Dataset ZIP

Click the upload button and select your `banking_dataset_*.zip` file.

In [ ]:
from google.colab import files
import zipfile
from pathlib import Path

print("📤 Click 'Choose Files' and upload your banking_dataset_*.zip")
uploaded = files.upload()

# Get the uploaded ZIP filename
zip_filename = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {zip_filename}")
print(f"   Size: {len(uploaded[zip_filename]) / 1024 / 1024:.1f} MB")

## 🗂️ Step 3: Prepare Dataset Structure

In [ ]:
import random
import shutil

print("🚀 Preparing dataset...")

# Create directories
base_path = Path('dataset')
dirs = {
    'train_images': base_path / 'images' / 'train',
    'train_labels': base_path / 'labels' / 'train',
    'val_images': base_path / 'images' / 'val',
    'val_labels': base_path / 'labels' / 'val',
    'test_images': base_path / 'images' / 'test',
    'test_labels': base_path / 'labels' / 'test',
}

for dir_path in dirs.values():
    dir_path.mkdir(parents=True, exist_ok=True)

print("✅ Created directory structure")

# Extract ZIP
temp_extract = Path('temp_extract')
temp_extract.mkdir(exist_ok=True)

print(f"📦 Extracting {zip_filename}...")
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(temp_extract)

# Find extracted folders
images_dir = temp_extract / 'images'
annotations_dir = temp_extract / 'annotations'

# Get all image files
image_files = list(images_dir.glob('*.jpg'))
print(f"📸 Found {len(image_files)} images")

# Shuffle and split (80/15/5)
random.seed(42)
random.shuffle(image_files)

total = len(image_files)
train_split = int(0.8 * total)
val_split = int(0.95 * total)

train_files = image_files[:train_split]
val_files = image_files[train_split:val_split]
test_files = image_files[val_split:]

print(f"📊 Split: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")

# Copy files
def copy_split(files, img_dir, label_dir):
    for img_file in files:
        shutil.copy2(img_file, img_dir / img_file.name)
        label_file = annotations_dir / img_file.with_suffix('.txt').name
        if label_file.exists():
            shutil.copy2(label_file, label_dir / label_file.name)

print("📂 Copying files...")
copy_split(train_files, dirs['train_images'], dirs['train_labels'])
copy_split(val_files, dirs['val_images'], dirs['val_labels'])
copy_split(test_files, dirs['test_images'], dirs['test_labels'])

# Read class names
classes_file = temp_extract / 'classes.txt'
if classes_file.exists():
    with open(classes_file, 'r') as f:
        class_names = [line.strip() for line in f.readlines()]
else:
    class_names = ['person', 'car', 'truck', 'handbag', 'backpack', 'bottle', 'cell phone']

print(f"🏷️  Classes: {', '.join(class_names)}")

# Create dataset.yaml
yaml_content = f"""path: /content/dataset
train: images/train
val: images/val
test: images/test

names:
"""

for idx, class_name in enumerate(class_names):
    yaml_content += f"  {idx}: {class_name}\n"

yaml_path = base_path / 'dataset.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"✅ Created {yaml_path}")

# Clean up
shutil.rmtree(temp_extract)
os.remove(zip_filename)

print("\n" + "="*60)
print("✨ Dataset preparation complete!")
print("="*60)
print(f"📁 Dataset location: {base_path.absolute()}")
print(f"📊 Total: {total} images")
print(f"🏷️  Classes: {len(class_names)}")
print("="*60)

## 🎯 Step 4: Train YOLOv8 Model

**Model sizes:**
- `yolov8n.pt` - Nano (fastest, ~2M params)
- `yolov8s.pt` - Small (balanced, ~9M params) ⭐ **Recommended**
- `yolov8m.pt` - Medium (~20M params)
- `yolov8l.pt` - Large (~43M params)

Training takes ~15-30 minutes with free GPU.

In [ ]:
# Training configuration
MODEL_SIZE = 's'  # Change to 'n', 'm', or 'l' if desired
EPOCHS = 100       # Number of training epochs
BATCH_SIZE = 16    # Batch size (reduce if out of memory)
IMG_SIZE = 640     # Input image size

print("🎯 Training Configuration:")
print(f"   Model: YOLOv8{MODEL_SIZE.upper()}")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"   Device: GPU ({torch.cuda.get_device_name(0)})")
print(f"\n⏱️  Estimated time: ~{EPOCHS * 0.2:.0f} minutes")
print("\n" + "="*60)
print("🎬 Starting training...")
print("="*60 + "\n")

In [ ]:
# Load pretrained model
model = YOLO(f'yolov8{MODEL_SIZE}.pt')

# Train the model
results = model.train(
    data='dataset/dataset.yaml',
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=0,
    project='runs/train',
    name='banking_detection',
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42,
    patience=50,
    save=True,
    save_period=10,
    amp=True,
    # Data augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
)

print("\n" + "="*60)
print("✅ Training completed!")
print("="*60)

## 📊 Step 5: Validate Model & View Results

In [ ]:
# Validate on test set
print("📊 Running validation...")
metrics = model.val()

print(f"\n🎯 Results:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")
print(f"   Precision: {metrics.box.mp:.3f}")
print(f"   Recall: {metrics.box.mr:.3f}")

# Display training results
from IPython.display import Image, display

print("\n📈 Training Results:")

results_dir = Path('runs/train/banking_detection')

# Show confusion matrix
if (results_dir / 'confusion_matrix.png').exists():
    print("\nConfusion Matrix:")
    display(Image(filename=str(results_dir / 'confusion_matrix.png')))

# Show results curves
if (results_dir / 'results.png').exists():
    print("\nTraining Curves:")
    display(Image(filename=str(results_dir / 'results.png')))

# Show sample predictions
if (results_dir / 'val_batch0_pred.jpg').exists():
    print("\nSample Predictions:")
    display(Image(filename=str(results_dir / 'val_batch0_pred.jpg')))

## 🧪 Step 6: Test Model on Sample Images

In [ ]:
# Load best model
best_model = YOLO('runs/train/banking_detection/weights/best.pt')

# Get a random test image
test_images = list(Path('dataset/images/test').glob('*.jpg'))
if test_images:
    test_img = random.choice(test_images)
    
    print(f"🧪 Testing on: {test_img.name}")
    
    # Run inference
    results = best_model(test_img)
    
    # Display results
    for result in results:
        result_img = result.plot()
        import cv2
        from matplotlib import pyplot as plt
        
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title('Model Prediction')
        plt.show()
        
        # Print detections
        boxes = result.boxes
        if len(boxes) > 0:
            print(f"\n✅ Found {len(boxes)} objects:")
            for box in boxes:
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                print(f"   - {result.names[cls]}: {conf:.2%}")
        else:
            print("\n⚠️  No objects detected in this image")

## 📥 Step 7: Download Trained Model

In [ ]:
# Create ZIP with model and results
import shutil

print("📦 Packaging model for download...")

# Create output directory
output_dir = Path('trained_model')
output_dir.mkdir(exist_ok=True)

# Copy model files
weights_dir = Path('runs/train/banking_detection/weights')
shutil.copy2(weights_dir / 'best.pt', output_dir / 'banking_model_best.pt')
shutil.copy2(weights_dir / 'last.pt', output_dir / 'banking_model_last.pt')

# Copy results
results_dir = Path('runs/train/banking_detection')
for img_file in ['confusion_matrix.png', 'results.png', 'val_batch0_pred.jpg']:
    src = results_dir / img_file
    if src.exists():
        shutil.copy2(src, output_dir / img_file)

# Create README
readme = f"""# Banking Detection Model

Trained: {pd.Timestamp.now()}

## Model Details
- Architecture: YOLOv8{MODEL_SIZE.upper()}
- Training epochs: {EPOCHS}
- mAP50: {metrics.box.map50:.3f}
- mAP50-95: {metrics.box.map:.3f}

## Files
- `banking_model_best.pt` - Best model checkpoint (use this!)
- `banking_model_last.pt` - Last epoch checkpoint
- `confusion_matrix.png` - Model performance visualization
- `results.png` - Training curves

## Usage

```python
from ultralytics import YOLO

# Load model
model = YOLO('banking_model_best.pt')

# Run inference
results = model('image.jpg')

# Process results
for result in results:
    boxes = result.boxes
    for box in boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        print(f"{{result.names[cls]}}: {{conf:.2%}}")
```

## Classes
{chr(10).join([f"{i}. {name}" for i, name in enumerate(class_names)])}
"""

with open(output_dir / 'README.md', 'w') as f:
    f.write(readme)

# Create ZIP
import pandas as pd
shutil.make_archive('banking_model', 'zip', output_dir)

print("✅ Model packaged successfully!")
print("\n📥 Downloading...")

# Download
files.download('banking_model.zip')

print("\n" + "="*60)
print("🎉 Training complete!")
print("="*60)
print("\n📦 Downloaded: banking_model.zip")
print("\n💡 Next steps:")
print("   1. Extract the ZIP file")
print("   2. Use 'banking_model_best.pt' in your app")
print("   3. Replace COCO-SSD with this custom model")
print("="*60)

---

## 🚀 Next Steps

1. **Extract downloaded ZIP**
2. **Integrate model into your app** - I'll help you replace COCO-SSD with your custom model
3. **Test on real ATM videos** - See improved detection!

Your custom model is now trained specifically on ATM/banking footage and will perform much better than the generic COCO-SSD model! 🎯